In [21]:
import time
import os

from absl import logging

import jax
import jax.numpy as jnp
from jax import random, vmap
from jax.tree_util import tree_map

import numpy as np
import scipy.io
import ml_collections
# import wandb

from jaxpi.samplers import UniformSampler
from jaxpi.logging import Logger
from jaxpi.utils import save_checkpoint

import models as models
import jaxpi.archs as arch
from utils import get_dataset

import importlib

In [22]:
import configs.pirate as pirate

config = pirate.get_config()

u_ref, v_ref, t_star, x_star, y_star, b1, b2, c1, c2, eps1, eps2 = get_dataset(
        config.time_fraction
    )

# Initial condition of the first time window
u0 = u_ref[0, :, :]
v0 = v_ref[0, :, :]

# Get the time domain for each time window
num_time_steps = len(t_star) // config.training.num_time_windows
t = t_star[:num_time_steps]

# Define the time and space domain
dt = t[1] - t[0]
t0 = t[0]
t1 = t[-1] + 2 * dt
# cover the start point of the next time window, which is t_star[num_time_steps]

x0 = x_star[0]
x1 = x_star[-1]

y0 = y_star[0]
y1 = y_star[-1]

dom = jnp.array([[t0, t1], [x0, x1], [y0, y1]])

In [74]:
import jaxpi.models as jm

importlib.reload(jm)
importlib.reload(models)
importlib.reload(arch)

model = models.LotteryTicketGreyScott(
            config, t, x_star, y_star, u0, v0, b1, b2, c1, c2, eps1, eps2
)

model.state, model.mask = model.prune_by_percentile(model.state, model.mask, 20)
model.state, model.mask = model.original_initialiaztion(model.state, model.mask)